In [14]:
import pandas as pd
import numpy as np
from os import listdir

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
import random
import math

In [15]:
def HeaderMaker(PatientFile):
    df_cleaned = pd.read_csv('/data/ortho/' + PatientFile, header=None)

    df_cleaned = df_cleaned.rename(columns={0: "thorax_r_x", 1: "thorax_r_y", 2: "thorax_r_z"})
    df_cleaned = df_cleaned.rename(columns={3: "clavicula_r_x", 4: "clavicula_r_y", 5: "clavicula_r_z"})
    df_cleaned = df_cleaned.rename(columns={6: "scapula_r_x", 7: "scapula_r_y", 8: "scapula_r_z"})
    df_cleaned = df_cleaned.rename(columns={9: "humerus_r_x", 10: "humerus_r_y", 11: "humerus_r_z"})
    df_cleaned = df_cleaned.rename(columns={12: "ellebooghoek_r"})
    df_cleaned = df_cleaned.rename(columns={15: "thorax_l_x", 16: "thorax_l_y", 17: "thorax_l_z"})
    df_cleaned = df_cleaned.rename(columns={18: "clavicula_l_x", 19: "clavicula_l_y", 20: "clavicula_l_z"})
    df_cleaned = df_cleaned.rename(columns={21: "scapula_l_x", 22: "scapula_l_y", 23: "scapula_l_z"})
    df_cleaned = df_cleaned.rename(columns={24: "humerus_l_x", 25: "humerus_l_y", 26: "humerus_l_z"})
    df_cleaned = df_cleaned.rename(columns={27: "ellebooghoek_l"})
    
    return df_cleaned

In [16]:

Patients = {}
testdataurl = '/data/ortho/'
for PatientFile in listdir(testdataurl):
    if ('meting' not in PatientFile):
        print('Skipping: %s' % PatientFile)
        # Skip this iteration
        continue
    Splitted = PatientFile.split('_')
    Patient = Splitted[0] + '_' + Splitted[1]

    Data = HeaderMaker(PatientFile)
    MaxDict = Data.abs().max().to_dict() # .abs() for absolute numbers
    
    if(Patient not in Patients.keys()):
        Patients[Patient] = MaxDict
        Patients[Patient]['GroundTruth'] = Splitted[0].split('Cat')[1]
        Patients[Patient]['Bias'] = 0
        continue
    
    for key in MaxDict.keys():
        Patients[Patient][key]= max(Patients[Patient][key],MaxDict[key])
        

    
    
            
            
            
            

Skipping: Cleaned Train
Skipping: StudentNummers
Skipping: testset
Skipping: AllPatients.csv
Skipping: codevanjurriaan.zip
Skipping: TestDataAllPatients.csv


In [17]:
df = pd.DataFrame.from_dict(Patients, orient='index')
# print(df.columns.values)
# df = df[['Cat2' in index or 'Cat3' in index for index in df.index]] # Welke colommen wil je houden

# df['TrueFalse'] = ['2' == i for i in df['GroundTruth']]

# print(df.head())

In [18]:
Xcolumns = ['Bias']
Xcolumns.extend(['thorax_r_x', 'thorax_r_y', 'thorax_r_z', 'clavicula_r_x', 'clavicula_r_y',
 'clavicula_r_z', 'scapula_r_x', 'scapula_r_y', 'scapula_r_z', 'humerus_r_x',
 'humerus_r_y', 'humerus_r_z', 'ellebooghoek_r', 'thorax_l_x',
 'thorax_l_y', 'thorax_l_z', 'clavicula_l_x', 'clavicula_l_y', 'clavicula_l_z',
 'scapula_l_x', 'scapula_l_y', 'scapula_l_z', 'humerus_l_x', 'humerus_l_y',
 'humerus_l_z', 'ellebooghoek_l'])

# X = df[Xcolumns]
# y = df[['GroundTruth']]




In [19]:
print(df['GroundTruth'].unique())

['1' '2' '3' '4']


In [20]:
def Patient_test_split(dataframe, xcol, ycol, Percentage):
    allIndex = np.unique(df.index.tolist()) # Get all unique patients id's
    random.seed(2) # Set random seeed so the answer is the same
    
    PercentageIndex = []
    for i in df['GroundTruth'].unique(): # Cycle through each categorie (only cycles through the ones that are present)
        CatPatients = np.array([i.split('_')[0].split('Cat')[1] for i in allIndex])
        CatPatients = CatPatients[CatPatients==i]
        
        AmountItems = len(CatPatients)
        AmountRandom = math.floor(AmountItems*Percentage)
        PercentageIndex.extend(random.sample(list(CatPatients), AmountRandom))        
        
    AmountItems = len(allIndex)
    AmountRandom = math.floor(AmountItems*Percentage)
    
    PercentageIndex = random.sample(list(allIndex), AmountRandom)
    
    Percentagedf = df[xcol][df.index.isin(PercentageIndex)]
    Percentagey = df[ycol][df.index.isin(PercentageIndex)]
    
    Testdf = df[xcol][~df.index.isin(PercentageIndex)]
    Testy = df[ycol][~df.index.isin(PercentageIndex)]
    print('Final Percentage test: %s' % (len(Testdf)/len(Percentagedf)))
    return (Percentagedf, Testdf, Percentagey, Testy)

X_train, X_test, y_train, y_test = Patient_test_split(df, Xcolumns, 'GroundTruth', 0.85)

Final Percentage test: 0.1797752808988764


In [21]:
print(len(X_train), len(X_test))

89 16


In [22]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [23]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [24]:
y_pred = lr.predict(X_test)
print(lr.coef_[0])
print(metrics.accuracy_score(y_test, y_pred))
print(y_test)

[ 0.00000000e+00 -8.00027109e-02 -2.49804728e-01 -3.72342310e-01
 -5.80912486e-02  7.14597311e-01 -2.61063912e-01  3.65779458e-01
 -1.67719673e-01  2.63473089e-01 -2.27179261e-01  4.07329064e-01
  1.17405409e-01  2.43475303e-05 -8.00027109e-02 -2.49804728e-01
 -3.72342310e-01  1.76049977e-01 -4.08428059e-01  1.84428644e-01
 -6.01074676e-01 -3.00158478e-01 -7.32277107e-01 -2.78170215e-01
 -1.61394384e-01  2.85625411e-01  2.02481113e-04]
0.625
Cat1_pat12    1
Cat1_pat3     1
Cat1_pat7     1
Cat2_pat1     2
Cat2_pat10    2
Cat2_pat12    2
Cat2_pat20    2
Cat2_pat26    2
Cat2_pat31    2
Cat2_pat34    2
Cat2_pat6     2
Cat3_pat15    3
Cat3_pat20    3
Cat3_pat25    3
Cat3_pat26    3
Cat3_pat37    3
Name: GroundTruth, dtype: object


In [25]:
from sklearn.metrics import mean_squared_error, \
recall_score, precision_score, accuracy_score, confusion_matrix

y_true = y_test
y_pred =lr.predict(X_test)

print(metrics.classification_report(y_true, y_pred, digits=3))

              precision    recall  f1-score   support

           1      0.500     0.667     0.571         3
           2      1.000     0.500     0.667         8
           3      0.571     0.800     0.667         5
           4      0.000     0.000     0.000         0

   micro avg      0.625     0.625     0.625        16
   macro avg      0.518     0.492     0.476        16
weighted avg      0.772     0.625     0.649        16



/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [26]:
print(df)

            thorax_r_x  thorax_r_y  thorax_r_z  clavicula_r_x  clavicula_r_y  \
Cat1_pat1    13.663678    9.903709   11.123922      64.694994      59.705033   
Cat1_pat10   12.806170    7.593816    9.302323      54.298075      44.600980   
Cat1_pat11    4.897893    6.773511    3.735479      56.065497      33.784876   
Cat1_pat12   49.245791    7.314965    8.305846      61.410984      20.651558   
Cat1_pat13   30.173814    8.499073    7.535154      48.753768      37.444309   
Cat1_pat14   11.258897    7.298017    7.133124      62.063262      34.299972   
Cat1_pat15   10.358437   10.047675    5.074372      59.905904      43.643171   
Cat1_pat16   44.385380    8.091777   11.688101      50.551228      33.370916   
Cat1_pat17    8.925056    6.575340    6.089936      70.999833      32.036554   
Cat1_pat18   24.864544   15.877012   10.475858      67.200225      49.649160   
Cat1_pat19   14.816584    7.588055    7.960695      56.279465      37.832953   
Cat1_pat2    20.354050   10.399477    6.